In [204]:
import os
os.chdir("/home/jardim/dev/tutoring-app-data-pipeline")
print(os.getcwd())

import modules.datagen as gen
import modules.bq_first_loader as bq
import importlib
from datetime import datetime
importlib.reload(gen)
importlib.reload(bq)

/home/jardim/dev/tutoring-app-data-pipeline


<module 'modules.bq_first_loader' from '/home/jardim/dev/tutoring-app-data-pipeline/modules/bq_first_loader.py'>

## Generating Mock Data For the First Load

In [198]:
students = gen.StudentsGenerator(1000)
subjects = gen.SubjectsGenerator()
tutors = gen.TutorsGenerator(100, subjects)
work_shifts = tutors.get_work_shifts()
sessions = gen.SessionsGenerator(students, subjects, tutors)
sessions.generate_sessions_for_date_range('2022-10-10', '2022-10-12')
sessions.process('50S')

## First Load to BigQuery

In [205]:
loader = bq.BQFirstLoader()
loader.load_df_to_table(students.get_students(), 'students')
loader.load_df_to_table(subjects.get_subjects(), 'subjects')
loader.load_df_to_table(tutors.get_work_shifts(), 'work_shifts')
loader.load_df_to_table(tutors.get_tutors(), 'tutors')
loader.load_df_to_table(sessions.get_sessions(), 'sessions')

Connecting to BQ.....  - Connected!
Loaded 1000 rows and 4 columns to jardim-data.dl_tutoring.students
Loaded 4 rows and 5 columns to jardim-data.dl_tutoring.subjects
Loaded 3 rows and 7 columns to jardim-data.dl_tutoring.work_shifts
Loaded 100 rows and 9 columns to jardim-data.dl_tutoring.tutors
Loaded 364 rows and 8 columns to jardim-data.dl_tutoring.sessions


## Generating Streaming Data

In [206]:
sessions.generate_sessions_for_date_range('2022-10-13', '2022-10-14')

In [ ]:
sessions.process('50S')

In [211]:
sessions._sessions#.head(1).to_json()

,session_id,started_at,student_id,subject_id,duration_min,status,tutor_id,connected_at,finished_at
0,100364,2022-10-13 07:14:46.551973,100921,100,24.878436,Queued,None,NaT,NaT
1,100365,2022-10-13 07:18:12.580694,100724,103,4.426739,Queued,None,NaT,NaT
2,100366,2022-10-13 07:21:26.109848,100189,101,16.557085,Queued,None,NaT,NaT
3,100367,2022-10-13 07:30:17.999227,100272,101,7.785882,Queued,None,NaT,NaT
4,100368,2022-10-13 07:38:38.846416,100165,100,9.880933,Queued,None,NaT,NaT
...,...,...,...,...,...,...,...,...,...
149,100513,2022-10-14 21:15:54.249281,100745,100,18.081667,Queued,None,NaT,NaT
150,100514,2022-10-14 21:54:02.350243,100600,100,25.452691,Queued,None,NaT,NaT
151,100515,2022-10-14 22:13:33.259774,100443,103,10.396263,Queued,None,NaT,NaT
152,100516,2022-10-14 22:21:19.782460,100233,101,10.425108,Queued,None,NaT,NaT
